## 🧠 Pretrained ResNet18
- 🔧 필요한 import 추가

```python
from torchvision.models import resnet18
```

- 🧠 모델 생성 함수 만들기 (사전학습 가중치 사용)

```python
def build_pretrained_resnet18(num_classes=10):
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    # 선택: CIFAR에 맞게 conv1 수정 (안 해도 동작은 함)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()

    return model.to(DEVICE)
```

- 🧠 전이학습 옵션

```python
# 1. 모델 정의
model = resnet18(pretrained=True)

# 2. 출력 레이어 수정 (1000 → 10)
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

# 3. 파라미터 학습 여부 설정
for param in model.parameters():
    param.requires_grad = False  # 전체 동결 (freeze)

for param in model.fc.parameters():
    param.requires_grad = True   # fc만 학습 가능

# 4. 모델 디바이스로 이동
model = model.to(DEVICE)

```

In [ ]:
# !pip install torchinfo

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import os
import json
import platform
import pkg_resources
from sklearn.metrics import precision_score, recall_score, f1_score
from torchinfo import summary
from torchvision.models import resnet18

# GPU 설정
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 하이퍼파라미터 설정
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-5
DROPOUT_RATE = 0.3
INPUT_SIZE = 3 * 32 * 32
NUM_CLASSES = 10
MODEL_PATH = './best_model.pth'
PATIENCE = 3
SPLIT_RATIO = 0.8

# CIFAR10 데이터셋의 평균 및 표준편차 (정규화 기준)
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

# 학습 데이터 증강 설정 (후반 학습용)
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 테스트 데이터 변환 (데이터 증강 없이 정규화만 적용)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 데이터셋 로드
full_train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root="./data", train=False, transform=transform_test, download=True)

# 학습 및 검증 데이터셋 분리
train_size = int(SPLIT_RATIO * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
val_dataset.dataset.transform = transform_test

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 모델 불러오기 (ResNet18)
def build_pretrained_resnet18(num_classes=10):
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    # 선택: CIFAR에 맞게 conv1 수정 (안 해도 동작은 함)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()

    # 파라미터 학습 여부 설정
    for param in model.parameters():
        param.requires_grad = False  # 전체 동결 (freeze)

    for param in model.fc.parameters():
        param.requires_grad = True   # fc만 학습 가능

    return model.to(DEVICE)

# 모델 초기화
model = build_pretrained_resnet18(num_classes=NUM_CLASSES)

# summary 호출
print(summary(model, input_size=(BATCH_SIZE, 3, 32, 32)))

# 모델 불러오기
def load_model():
    if os.path.exists(MODEL_PATH):
        model.load_state_dict(torch.load(MODEL_PATH))
        print('Model loaded from checkpoint.')
    else:
        print('No checkpoint found, training from scratch.')
    return model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

best_val_acc = 0
patience_counter = 0

# 학습 함수 정의
def train():
    global best_val_acc, patience_counter
    model.train()
    for epoch in range(EPOCHS):

        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        scheduler.step()

        avg_loss = running_loss / len(train_loader)
        val_loss, val_acc = validate()

        print(f"Epoch [{epoch+1}/{EPOCHS}], Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:  # Early Stopping 체크
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"New Best Model Saved! Validation Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print("Early stopping triggered.")
                break

def validate():
    model.eval()
    correct = 0
    total = 0
    val_loss_total = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss_total += loss.item()

            _, predicted = torch.max(outputs, 1) # 반환:(최댓값, 인덱스)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss_total / len(val_loader)
    val_acc = 100 * correct / total

    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"Validation - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    return val_loss, val_acc


def test():
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f'Test Accuracy: {accuracy:.2f}%')
    print(f'Test Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')


# 최적의 하이퍼파라미터 정보와 환경 정보 저장 함수
def save_study_info(filename="best_study_info.json"):

    # Python 및 라이브러리 정보 가져오기
    python_version = platform.python_version()
    platform_info = platform.platform()
    installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}

    # 데이터셋 정보 수집
    dataset_name = 'CIFAR10'  # 현재 사용 중인 데이터셋 이름
    dataset_info = {
        "dataset_name": dataset_name,
        "train_size": len(train_dataset),
        "validation_size": len(val_dataset),
        "test_size": len(test_dataset),
        "image_size": (32, 32),  # CIFAR-10 이미지 크기
        "num_classes": NUM_CLASSES
    }

    # 모델 정보 수집
    model_info = {
        "model_name": model.__class__.__name__,
        "model_structure": str(model),
        "input_size": INPUT_SIZE,
        "num_classes": NUM_CLASSES
    }

    # 정보 저장
    data = {
        "python_version": python_version,
        "platform_info": platform_info,
        "installed_packages": installed_packages,
        "dataset_info": dataset_info,
        "model_info": model_info
    }

    with open(filename, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Best study info saved to {filename}")


if __name__ == "__main__":
    train()
    # load_model()
    test()

    # 최적의 정보 저장
    save_study_info()

<ipython-input-1-acf9fcea670d>:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
100%|██████████| 170M/170M [00:02<00:00, 68.6MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████|

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [128, 10]                 --
├─Conv2d: 1-1                            [128, 64, 32, 32]         (1,728)
├─BatchNorm2d: 1-2                       [128, 64, 32, 32]         (128)
├─ReLU: 1-3                              [128, 64, 32, 32]         --
├─Identity: 1-4                          [128, 64, 32, 32]         --
├─Sequential: 1-5                        [128, 64, 32, 32]         --
│    └─BasicBlock: 2-1                   [128, 64, 32, 32]         --
│    │    └─Conv2d: 3-1                  [128, 64, 32, 32]         (36,864)
│    │    └─BatchNorm2d: 3-2             [128, 64, 32, 32]         (128)
│    │    └─ReLU: 3-3                    [128, 64, 32, 32]         --
│    │    └─Conv2d: 3-4                  [128, 64, 32, 32]         (36,864)
│    │    └─BatchNorm2d: 3-5             [128, 64, 32, 32]         (128)
│    │    └─ReLU: 3-6                    [128, 64, 32, 32] 

## A. 베이스라인 구성 요약

| 항목 |	내용 |
|---|---|
|모델|pretrained ResNet18 + fc 수정|
|Optimizer|Adam|
|Learning Rate|	0.001 (고정)|
|Epochs|	10|
|데이터 증강|	기본 (Flip + Crop)|
|정규화|	없음 (기본 CrossEntropyLoss)|
|평가지표|accuracy, f1-score, precision, recall|

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from sklearn.metrics import f1_score, precision_score, recall_score
import csv
import os

# -----------------------------
# Config
# -----------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 10
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001
MODEL_PATH = "best_model.pth"
CSV_LOG = "experiment_results_A.csv"

# -----------------------------
# Data Transforms & Loaders
# -----------------------------
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform_test, download=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# -----------------------------
# Model Definition
# -----------------------------
def build_baseline_model():
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
    return model.to(DEVICE)

# -----------------------------
# Training Function
# -----------------------------
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# -----------------------------
# Evaluation Function
# -----------------------------
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = (torch.tensor(all_preds) == torch.tensor(all_labels)).float().mean().item() * 100
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

    return total_loss / len(loader), acc, f1, precision, recall

# -----------------------------
# Save CSV Log
# -----------------------------
def log_results(epoch, train_loss, test_loss, acc, f1, precision, recall):
    file_exists = os.path.isfile(CSV_LOG)
    with open(CSV_LOG, mode='a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(['epoch', 'train_loss', 'test_loss', 'test_acc', 'test_f1', 'test_precision', 'test_recall'])
        writer.writerow([epoch, train_loss, test_loss, acc, f1, precision, recall])

# -----------------------------
# Run Baseline Experiment
# -----------------------------
if __name__ == "__main__":
    model = build_baseline_model()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()

    best_f1 = 0.0

    for epoch in range(EPOCHS):
        train_loss = train(model, train_loader, optimizer, criterion)
        test_loss, test_acc, test_f1, test_precision, test_recall = evaluate(model, test_loader, criterion)

        print(f"[Epoch {epoch+1}/{EPOCHS}] "
              f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | "
              f"Acc: {test_acc:.2f}% | F1: {test_f1:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f}")

        log_results(epoch+1, train_loss, test_loss, test_acc, test_f1, test_precision, test_recall)

        if test_f1 > best_f1:
            best_f1 = test_f1
            torch.save(model.state_dict(), MODEL_PATH)
            print("✅ Best model saved!")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1/10] Train Loss: 1.0183 | Test Loss: 0.7654 | Acc: 74.02% | F1: 0.7408 | Precision: 0.7528 | Recall: 0.7402
Best model saved!
[Epoch 2/10] Train Loss: 0.7371 | Test Loss: 0.7276 | Acc: 75.77% | F1: 0.7539 | Precision: 0.7663 | Recall: 0.7577
Best model saved!
[Epoch 3/10] Train Loss: 0.6515 | Test Loss: 0.6653 | Acc: 77.02% | F1: 0.7701 | Precision: 0.7792 | Recall: 0.7702
Best model saved!
[Epoch 4/10] Train Loss: 0.6015 | Test Loss: 0.6450 | Acc: 78.28% | F1: 0.7842 | Precision: 0.7988 | Recall: 0.7828
Best model saved!
[Epoch 5/10] Train Loss: 0.5519 | Test Loss: 0.5892 | Acc: 79.65% | F1: 0.7983 | Precision: 0.8079 | Recall: 0.7965
Best model saved!
[Epoch 6/10] Train Loss: 0.5183 | Test Loss: 0.5563 | Acc: 80.73% | F1: 0.8067 | Precision: 0.8140 | Recall: 0.8073
Best model saved!
[Epoch 7/10] Train Loss: 0.5034 | Test Loss: 0.5774 | Acc: 79.97% | F1: 0.8010 | Precision: 0.8100 | Recall: 0.7997
[Epoch 8/10] Train Loss: 0.4777 | Test Loss: 0.5386 | Acc: 82.16% | F1: 0.8207 |

## B. OneCycleLR 적용

| 항목 |	내용 |
|---|---|
|모델|pretrained ResNet18 + fc 수정|
|Optimizer|Adam|
|Learning Rate|	OneCycleLR(max_lr=0.01)|
|Epochs|	10|
|데이터 증강|	기본 (Flip + Crop)|
|정규화|	없음 (기본 CrossEntropyLoss)|
|평가지표|accuracy, f1-score, precision, recall|

```
lr
│     ▲
│    /\\           max_lr = 0.01
│   /  \\__        
│  /     \\     ↓ 감소구간
│_/       \\_____________ final_lr ≈ 1e-6
│
└────────────────────▶ steps
  ^      ^           ^
  |      |           |
  시작    최고점       종료
  ```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from sklearn.metrics import f1_score, precision_score, recall_score
import csv
import os

# -----------------------------
# Config
# -----------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 10
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001
MODEL_PATH = "best_model.pth"
CSV_LOG = "experiment_results_B.csv"
EXPERIMENT_NAME = "B_onecycle"

# -----------------------------
# Data Transforms & Loaders
# -----------------------------
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform_test, download=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# -----------------------------
# Model Definition
# -----------------------------
def build_baseline_model():
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
    return model.to(DEVICE)

# -----------------------------
# Training Function
# -----------------------------
def train(model, loader, optimizer, criterion, scheduler=None):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# -----------------------------
# Evaluation Function
# -----------------------------
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = (torch.tensor(all_preds) == torch.tensor(all_labels)).float().mean().item() * 100
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

    return total_loss / len(loader), acc, f1, precision, recall

# -----------------------------
# Save CSV Log
# -----------------------------
def log_results(epoch, train_loss, test_loss, acc, f1, precision, recall):
    file_exists = os.path.isfile(CSV_LOG)
    with open(CSV_LOG, mode='a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(['epoch', 'train_loss', 'test_loss', 'test_acc', 'test_f1', 'test_precision', 'test_recall'])
        writer.writerow([epoch, train_loss, test_loss, acc, f1, precision, recall])

# -----------------------------
# Run OneCycleLR Experiment (B)
# -----------------------------
if __name__ == "__main__":
    model = build_baseline_model()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=LEARNING_RATE, epochs=EPOCHS, steps_per_epoch=len(train_loader)
    )
    criterion = nn.CrossEntropyLoss()

    best_f1 = 0.0

    for epoch in range(EPOCHS):
        train_loss = train(model, train_loader, optimizer, criterion, scheduler)
        test_loss, test_acc, test_f1, test_precision, test_recall = evaluate(model, test_loader, criterion)

        print(f"[Epoch {epoch+1}/{EPOCHS}] "
              f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | "
              f"Acc: {test_acc:.2f}% | F1: {test_f1:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f}")

        log_results(epoch+1, train_loss, test_loss, test_acc, test_f1, test_precision, test_recall)

        if test_f1 > best_f1:
            best_f1 = test_f1
            torch.save(model.state_dict(), MODEL_PATH)
            print("✅ Best model saved!")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1/10] Train Loss: 1.3622 | Test Loss: 0.8592 | Acc: 70.50% | F1: 0.7028 | Precision: 0.7098 | Recall: 0.7050
✅ Best model saved!
[Epoch 2/10] Train Loss: 0.8192 | Test Loss: 0.7642 | Acc: 73.81% | F1: 0.7363 | Precision: 0.7452 | Recall: 0.7381
✅ Best model saved!
[Epoch 3/10] Train Loss: 0.7255 | Test Loss: 0.6921 | Acc: 76.64% | F1: 0.7617 | Precision: 0.7735 | Recall: 0.7664
✅ Best model saved!
[Epoch 4/10] Train Loss: 0.6532 | Test Loss: 0.6135 | Acc: 79.36% | F1: 0.7973 | Precision: 0.8096 | Recall: 0.7936
✅ Best model saved!
[Epoch 5/10] Train Loss: 0.5697 | Test Loss: 0.6204 | Acc: 79.63% | F1: 0.7982 | Precision: 0.8063 | Recall: 0.7963
✅ Best model saved!
[Epoch 6/10] Train Loss: 0.5071 | Test Loss: 0.5041 | Acc: 83.11% | F1: 0.8295 | Precision: 0.8305 | Recall: 0.8311
✅ Best model saved!
[Epoch 7/10] Train Loss: 0.4385 | Test Loss: 0.4667 | Acc: 84.12% | F1: 0.8403 | Precision: 0.8415 | Recall: 0.8412
✅ Best model saved!
[Epoch 8/10] Train Loss: 0.3681 | Test Loss: 0.4

## C. Mixup 적용

| 항목 |	내용 |
|---|---|
|모델|pretrained ResNet18 + fc 수정|
|Optimizer|Adam|
|Learning Rate|	OneCycleLR(max_lr=0.01)|
|Epochs|	10|
|데이터 증강|	기본 (Flip + Crop), mixup(alpha=0.2)|
|정규화|	없음 (기본 CrossEntropyLoss)|
|평가지표|accuracy, f1-score, precision, recall|

### 🌀 Mixup이란?
두 개의 이미지와 라벨을 선형 조합하여 모델에 학습시키는 방법.
- **"x = lam·x₁ + (1-lam)·x₂"**
- **"y = lam·y₁ + (1-lam)·y₂"**

```
입력 이미지 A (개)                   입력 이미지 B (고양이)
        │                                     │
        └────┬────lam────────┘
                  ▼
        섞인 이미지 (개 + 고양이)

정답 라벨: 개 70%, 고양이 30%
```

이렇게 학습하면:
- 결정 경계가 더 부드럽고 안정적
- 과적합 ↓, 일반화 ↑

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from sklearn.metrics import f1_score, precision_score, recall_score
import csv
import os
import random

# -----------------------------
# Config
# -----------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 10
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001
MAX_LR = 0.01
MODEL_PATH = "best_model.pth"
CSV_LOG = "experiment_results_C.csv"
EXPERIMENT_NAME = "C_mixup"
ALPHA = 0.2  # mixup alpha value

# -----------------------------
# Data Transforms & Loaders
# -----------------------------
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform_test, download=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# -----------------------------
# Model Definition
# -----------------------------
def build_baseline_model():
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
    return model.to(DEVICE)

# -----------------------------
# Mixup Function
# -----------------------------
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = torch.distributions.Beta(alpha, alpha).sample().item()
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(DEVICE)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# -----------------------------
# Training Function
# -----------------------------
def train(model, loader, optimizer, criterion, scheduler=None):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        x, y_a, y_b, lam = mixup_data(x, y, ALPHA)

        optimizer.zero_grad()
        outputs = model(x)
        loss = mixup_criterion(criterion, outputs, y_a, y_b, lam)
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# -----------------------------
# Evaluation Function
# -----------------------------
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = (torch.tensor(all_preds) == torch.tensor(all_labels)).float().mean().item() * 100
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

    return total_loss / len(loader), acc, f1, precision, recall

# -----------------------------
# Save CSV Log
# -----------------------------
def log_results(epoch, train_loss, test_loss, acc, f1, precision, recall):
    file_exists = os.path.isfile(CSV_LOG)
    with open(CSV_LOG, mode='a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(['epoch', 'train_loss', 'test_loss', 'test_acc', 'test_f1', 'test_precision', 'test_recall'])
        writer.writerow([epoch, train_loss, test_loss, acc, f1, precision, recall])

# -----------------------------
# Run Mixup Experiment (C)
# -----------------------------
if __name__ == "__main__":
    model = build_baseline_model()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=MAX_LR, epochs=EPOCHS, steps_per_epoch=len(train_loader)
    )
    criterion = nn.CrossEntropyLoss()

    best_f1 = 0.0

    for epoch in range(EPOCHS):
        train_loss = train(model, train_loader, optimizer, criterion, scheduler)
        test_loss, test_acc, test_f1, test_precision, test_recall = evaluate(model, test_loader, criterion)

        print(f"[Epoch {epoch+1}/{EPOCHS}] "
              f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | "
              f"Acc: {test_acc:.2f}% | F1: {test_f1:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f}")

        log_results(epoch+1, train_loss, test_loss, test_acc, test_f1, test_precision, test_recall)

        if test_f1 > best_f1:
            best_f1 = test_f1
            torch.save(model.state_dict(), MODEL_PATH)
            print("✅ Best model saved!")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1/10] Train Loss: 1.4111 | Test Loss: 1.2367 | Acc: 57.49% | F1: 0.5513 | Precision: 0.6117 | Recall: 0.5749
✅ Best model saved!
[Epoch 2/10] Train Loss: 1.4346 | Test Loss: 1.1424 | Acc: 62.21% | F1: 0.6093 | Precision: 0.6670 | Recall: 0.6221
✅ Best model saved!
[Epoch 3/10] Train Loss: 1.3829 | Test Loss: 1.1948 | Acc: 59.01% | F1: 0.5741 | Precision: 0.6248 | Recall: 0.5901
[Epoch 4/10] Train Loss: 1.2835 | Test Loss: 1.0320 | Acc: 65.40% | F1: 0.6483 | Precision: 0.6910 | Recall: 0.6540
✅ Best model saved!
[Epoch 5/10] Train Loss: 1.1021 | Test Loss: 0.8928 | Acc: 69.10% | F1: 0.6821 | Precision: 0.7107 | Recall: 0.6910
✅ Best model saved!
[Epoch 6/10] Train Loss: 1.0799 | Test Loss: 0.7293 | Acc: 76.16% | F1: 0.7591 | Precision: 0.7662 | Recall: 0.7616
✅ Best model saved!
[Epoch 7/10] Train Loss: 0.9969 | Test Loss: 0.6409 | Acc: 78.71% | F1: 0.7848 | Precision: 0.7935 | Recall: 0.7871
✅ Best model saved!
[Epoch 8/10] Train Loss: 0.9742 | Test Loss: 0.6048 | Acc: 80.65% | 

## D. Label Smoothing 적용

| 항목 |	내용 |
|---|---|
|모델|pretrained ResNet18 + fc 수정|
|Optimizer|Adam|
|Learning Rate|	OneCycleLR(max_lr=0.01)|
|Epochs|	10|
|데이터 증강|	기본 (Flip + Crop), mixup(alpha=0.2)|
|정규화|	Label Smoothing (기본 CrossEntropyLoss)|
|평가지표|accuracy, f1-score, precision, recall|

✅ Label Smoothing이란?
- 정답 라벨을 100% 확신하지 않도록 부드럽게 만드는 기법.

|클래스	|One-hot 라벨|	모델 출력|
|---|---|---|
|개|	1|	0.80|
|고양이	|0|	0.10|
|말	|0	|0.10|


|클래스|	Smoothed 라벨 (α=0.1)|
|---|---|
|개|	0.90|
|고양이	|0.05|
|말	|0.05|

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from sklearn.metrics import f1_score, precision_score, recall_score
import csv
import os

# -----------------------------
# Config
# -----------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 10
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001
MAX_LR = 0.01
MODEL_PATH = "best_model.pth"
CSV_LOG = "experiment_results_D.csv"
EXPERIMENT_NAME = "D_labelsmoothing"
LABEL_SMOOTHING = 0.1

# -----------------------------
# Data Transforms & Loaders
# -----------------------------
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform_test, download=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# -----------------------------
# Model Definition
# -----------------------------
def build_baseline_model():
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
    return model.to(DEVICE)

# -----------------------------
# Training Function
# -----------------------------
def train(model, loader, optimizer, criterion, scheduler=None):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# -----------------------------
# Evaluation Function
# -----------------------------
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = (torch.tensor(all_preds) == torch.tensor(all_labels)).float().mean().item() * 100
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

    return total_loss / len(loader), acc, f1, precision, recall

# -----------------------------
# Save CSV Log
# -----------------------------
def log_results(epoch, train_loss, test_loss, acc, f1, precision, recall):
    file_exists = os.path.isfile(CSV_LOG)
    with open(CSV_LOG, mode='a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(['epoch', 'train_loss', 'test_loss', 'test_acc', 'test_f1', 'test_precision', 'test_recall'])
        writer.writerow([epoch, train_loss, test_loss, acc, f1, precision, recall])

# -----------------------------
# Run Label Smoothing Experiment (D)
# -----------------------------
if __name__ == "__main__":
    model = build_baseline_model()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=MAX_LR, epochs=EPOCHS, steps_per_epoch=len(train_loader)
    )
    criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTHING)

    best_f1 = 0.0

    for epoch in range(EPOCHS):
        train_loss = train(model, train_loader, optimizer, criterion, scheduler)
        test_loss, test_acc, test_f1, test_precision, test_recall = evaluate(model, test_loader, criterion)

        print(f"[Epoch {epoch+1}/{EPOCHS}] "
              f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | "
              f"Acc: {test_acc:.2f}% | F1: {test_f1:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f}")

        log_results(epoch+1, train_loss, test_loss, test_acc, test_f1, test_precision, test_recall)

        if test_f1 > best_f1:
            best_f1 = test_f1
            torch.save(model.state_dict(), MODEL_PATH)
            print("✅ Best model saved!")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1/10] Train Loss: 1.3576 | Test Loss: 1.3693 | Acc: 63.95% | F1: 0.6413 | Precision: 0.6657 | Recall: 0.6395
✅ Best model saved!
[Epoch 2/10] Train Loss: 1.3486 | Test Loss: 3.9348 | Acc: 29.08% | F1: 0.2557 | Precision: 0.3846 | Recall: 0.2908
[Epoch 3/10] Train Loss: 1.4065 | Test Loss: 1.2390 | Acc: 68.35% | F1: 0.6796 | Precision: 0.6866 | Recall: 0.6835
✅ Best model saved!
[Epoch 4/10] Train Loss: 1.1620 | Test Loss: 1.1853 | Acc: 69.84% | F1: 0.7060 | Precision: 0.7312 | Recall: 0.6984
✅ Best model saved!
[Epoch 5/10] Train Loss: 1.0849 | Test Loss: 1.0826 | Acc: 74.36% | F1: 0.7431 | Precision: 0.7574 | Recall: 0.7436
✅ Best model saved!
[Epoch 6/10] Train Loss: 1.0195 | Test Loss: 1.0252 | Acc: 77.17% | F1: 0.7644 | Precision: 0.7809 | Recall: 0.7717
✅ Best model saved!
[Epoch 7/10] Train Loss: 0.9572 | Test Loss: 0.9657 | Acc: 79.43% | F1: 0.7919 | Precision: 0.7950 | Recall: 0.7943
✅ Best model saved!
[Epoch 8/10] Train Loss: 1.0420 | Test Loss: 0.9752 | Acc: 79.21% | 

## E. CutMix 적용

| 항목 |	내용 |
|---|---|
|모델|pretrained ResNet18 + fc 수정|
|Optimizer|Adam|
|Learning Rate|	OneCycleLR(max_lr=0.01)|
|Epochs|	10|
|데이터 증강|	기본 (Flip + Crop), CutMix(alpha=1.0)|
|정규화|	없음 (기본 CrossEntropyLoss)|
|평가지표|accuracy, f1-score, precision, recall|


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from sklearn.metrics import f1_score, precision_score, recall_score
import csv
import os
import numpy as np

# -----------------------------
# Config
# -----------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 10
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001
MAX_LR = 0.01
MODEL_PATH = "best_model.pth"
CSV_LOG = "experiment_results_E.csv"
EXPERIMENT_NAME = "E_cutmix"
CUTMIX_ALPHA = 1.0

# -----------------------------
# Data Transforms & Loaders
# -----------------------------
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform_test, download=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# -----------------------------
# CutMix Function
# -----------------------------
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# -----------------------------
# Model Definition
# -----------------------------
def build_baseline_model():
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
    return model.to(DEVICE)

# -----------------------------
# Training Function
# -----------------------------
def train(model, loader, optimizer, criterion, scheduler=None):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)

        indices = torch.randperm(x.size(0)).to(DEVICE)
        x2, y2 = x[indices], y[indices]
        lam = np.random.beta(CUTMIX_ALPHA, CUTMIX_ALPHA)

        bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
        x[:, :, bby1:bby2, bbx1:bbx2] = x2[:, :, bby1:bby2, bbx1:bbx2]

        # Adjust lambda based on the area of the cut region
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size(-1) * x.size(-2)))

        optimizer.zero_grad()
        outputs = model(x)
        loss = lam * criterion(outputs, y) + (1 - lam) * criterion(outputs, y2)
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# -----------------------------
# Evaluation Function
# -----------------------------
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = (torch.tensor(all_preds) == torch.tensor(all_labels)).float().mean().item() * 100
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

    return total_loss / len(loader), acc, f1, precision, recall

# -----------------------------
# Save CSV Log
# -----------------------------
def log_results(epoch, train_loss, test_loss, acc, f1, precision, recall):
    file_exists = os.path.isfile(CSV_LOG)
    with open(CSV_LOG, mode='a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(['epoch', 'train_loss', 'test_loss', 'test_acc', 'test_f1', 'test_precision', 'test_recall'])
        writer.writerow([epoch, train_loss, test_loss, acc, f1, precision, recall])

# -----------------------------
# Run CutMix Experiment (E)
# -----------------------------
if __name__ == "__main__":
    model = build_baseline_model()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=MAX_LR, epochs=EPOCHS, steps_per_epoch=len(train_loader)
    )
    criterion = nn.CrossEntropyLoss()

    best_f1 = 0.0

    for epoch in range(EPOCHS):
        train_loss = train(model, train_loader, optimizer, criterion, scheduler)
        test_loss, test_acc, test_f1, test_precision, test_recall = evaluate(model, test_loader, criterion)

        print(f"[Epoch {epoch+1}/{EPOCHS}] "
              f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | "
              f"Acc: {test_acc:.2f}% | F1: {test_f1:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f}")

        log_results(epoch+1, train_loss, test_loss, test_acc, test_f1, test_precision, test_recall)

        if test_f1 > best_f1:
            best_f1 = test_f1
            torch.save(model.state_dict(), MODEL_PATH)
            print("✅ Best model saved!")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1/10] Train Loss: 1.8268 | Test Loss: 1.2928 | Acc: 56.02% | F1: 0.5430 | Precision: 0.6090 | Recall: 0.5602
✅ Best model saved!
[Epoch 2/10] Train Loss: 1.8932 | Test Loss: 1.6965 | Acc: 38.60% | F1: 0.3407 | Precision: 0.4789 | Recall: 0.3860
[Epoch 3/10] Train Loss: 1.9342 | Test Loss: 1.4444 | Acc: 49.72% | F1: 0.4828 | Precision: 0.5360 | Recall: 0.4972
[Epoch 4/10] Train Loss: 1.7837 | Test Loss: 1.1898 | Acc: 59.85% | F1: 0.5901 | Precision: 0.6144 | Recall: 0.5985
✅ Best model saved!
[Epoch 5/10] Train Loss: 1.6941 | Test Loss: 1.0709 | Acc: 63.83% | F1: 0.6360 | Precision: 0.6563 | Recall: 0.6383
✅ Best model saved!
[Epoch 6/10] Train Loss: 1.6412 | Test Loss: 1.0488 | Acc: 67.15% | F1: 0.6705 | Precision: 0.6914 | Recall: 0.6715
✅ Best model saved!
[Epoch 7/10] Train Loss: 1.5801 | Test Loss: 0.8864 | Acc: 71.05% | F1: 0.7101 | Precision: 0.7184 | Recall: 0.7105
✅ Best model saved!
[Epoch 8/10] Train Loss: 1.5559 | Test Loss: 0.8280 | Acc: 73.12% | F1: 0.7298 | Precisi

## F. AdamW 적용

| 항목 |	내용 |
|---|---|
|모델|pretrained ResNet18 + fc 수정|
|Optimizer|AdamW|
|Learning Rate|	OneCycleLR(max_lr=0.01)|
|Epochs|	10|
|데이터 증강|	기본 (Flip + Crop)|
|정규화|	Label Smoothing (기본 CrossEntropyLoss)|
|평가지표|accuracy, f1-score, precision, recall|

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from sklearn.metrics import f1_score, precision_score, recall_score
import csv
import os
import numpy as np

# -----------------------------
# Config
# -----------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 10
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001
MAX_LR = 0.01
MODEL_PATH = "best_model.pth"
CSV_LOG = "experiment_results_F.csv"
EXPERIMENT_NAME = "F_adamw"

# -----------------------------
# Data Transforms & Loaders
# -----------------------------
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform_test, download=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# -----------------------------
# Model Definition
# -----------------------------
def build_baseline_model():
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
    return model.to(DEVICE)

# -----------------------------
# Training Function
# -----------------------------
def train(model, loader, optimizer, criterion, scheduler=None):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# -----------------------------
# Evaluation Function
# -----------------------------
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = (torch.tensor(all_preds) == torch.tensor(all_labels)).float().mean().item() * 100
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

    return total_loss / len(loader), acc, f1, precision, recall

# -----------------------------
# Save CSV Log
# -----------------------------
def log_results(epoch, train_loss, test_loss, acc, f1, precision, recall):
    file_exists = os.path.isfile(CSV_LOG)
    with open(CSV_LOG, mode='a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(['epoch', 'train_loss', 'test_loss', 'test_acc', 'test_f1', 'test_precision', 'test_recall'])
        writer.writerow([epoch, train_loss, test_loss, acc, f1, precision, recall])

# -----------------------------
# Run AdamW Optimizer Experiment (F)
# -----------------------------
if __name__ == "__main__":
    model = build_baseline_model()
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=MAX_LR, epochs=EPOCHS, steps_per_epoch=len(train_loader)
    )
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    best_f1 = 0.0

    for epoch in range(EPOCHS):
        train_loss = train(model, train_loader, optimizer, criterion, scheduler)
        test_loss, test_acc, test_f1, test_precision, test_recall = evaluate(model, test_loader, criterion)

        print(f"[Epoch {epoch+1}/{EPOCHS}] "
              f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | "
              f"Acc: {test_acc:.2f}% | F1: {test_f1:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f}")

        log_results(epoch+1, train_loss, test_loss, test_acc, test_f1, test_precision, test_recall)

        if test_f1 > best_f1:
            best_f1 = test_f1
            torch.save(model.state_dict(), MODEL_PATH)
            print("✅ Best model saved!")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1/10] Train Loss: 1.3584 | Test Loss: 1.5930 | Acc: 54.48% | F1: 0.5469 | Precision: 0.6637 | Recall: 0.5448
✅ Best model saved!
[Epoch 2/10] Train Loss: 1.3926 | Test Loss: 1.4346 | Acc: 59.31% | F1: 0.5895 | Precision: 0.5964 | Recall: 0.5931
✅ Best model saved!
[Epoch 3/10] Train Loss: 1.2823 | Test Loss: 1.1763 | Acc: 70.65% | F1: 0.7077 | Precision: 0.7196 | Recall: 0.7065
✅ Best model saved!
[Epoch 4/10] Train Loss: 1.1429 | Test Loss: 1.2107 | Acc: 70.34% | F1: 0.6991 | Precision: 0.7245 | Recall: 0.7034
[Epoch 5/10] Train Loss: 1.0784 | Test Loss: 1.1105 | Acc: 73.47% | F1: 0.7337 | Precision: 0.7560 | Recall: 0.7347
✅ Best model saved!
[Epoch 6/10] Train Loss: 1.0152 | Test Loss: 0.9805 | Acc: 78.77% | F1: 0.7851 | Precision: 0.7880 | Recall: 0.7877
✅ Best model saved!
[Epoch 7/10] Train Loss: 0.9585 | Test Loss: 0.9357 | Acc: 80.82% | F1: 0.8093 | Precision: 0.8116 | Recall: 0.8082
✅ Best model saved!
[Epoch 8/10] Train Loss: 0.8931 | Test Loss: 0.8930 | Acc: 82.83% | 

## G. AutoAugment 적용

| 항목 |	내용 |
|---|---|
|모델|pretrained ResNet18 + fc 수정|
|Optimizer|AdamW|
|Learning Rate|	OneCycleLR(max_lr=0.01)|
|Epochs|	10|
|데이터 증강|	AutoAugment(CIFAR10) |
|정규화|	Label Smoothing (기본 CrossEntropyLoss)|
|평가지표|accuracy, f1-score, precision, recall|

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from sklearn.metrics import f1_score, precision_score, recall_score
import csv
import os
import numpy as np

# -----------------------------
# Config
# -----------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 10
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001
MAX_LR = 0.01
MODEL_PATH = "best_model.pth"
CSV_LOG = "experiment_results_G.csv"
EXPERIMENT_NAME = "G_autoaugment"

# -----------------------------
# Data Transforms & Loaders
# -----------------------------
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform_test, download=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# -----------------------------
# Model Definition
# -----------------------------
def build_baseline_model():
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
    return model.to(DEVICE)

# -----------------------------
# Training Function
# -----------------------------
def train(model, loader, optimizer, criterion, scheduler=None):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# -----------------------------
# Evaluation Function
# -----------------------------
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = (torch.tensor(all_preds) == torch.tensor(all_labels)).float().mean().item() * 100
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

    return total_loss / len(loader), acc, f1, precision, recall

# -----------------------------
# Save CSV Log
# -----------------------------
def log_results(epoch, train_loss, test_loss, acc, f1, precision, recall):
    file_exists = os.path.isfile(CSV_LOG)
    with open(CSV_LOG, mode='a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(['epoch', 'train_loss', 'test_loss', 'test_acc', 'test_f1', 'test_precision', 'test_recall'])
        writer.writerow([epoch, train_loss, test_loss, acc, f1, precision, recall])

# -----------------------------
# Run AdamW Optimizer Experiment (F)
# -----------------------------
if __name__ == "__main__":
    model = build_baseline_model()
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=MAX_LR, epochs=EPOCHS, steps_per_epoch=len(train_loader)
    )
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    best_f1 = 0.0

    for epoch in range(EPOCHS):
        train_loss = train(model, train_loader, optimizer, criterion, scheduler)
        test_loss, test_acc, test_f1, test_precision, test_recall = evaluate(model, test_loader, criterion)

        print(f"[Epoch {epoch+1}/{EPOCHS}] "
              f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | "
              f"Acc: {test_acc:.2f}% | F1: {test_f1:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f}")

        log_results(epoch+1, train_loss, test_loss, test_acc, test_f1, test_precision, test_recall)

        if test_f1 > best_f1:
            best_f1 = test_f1
            torch.save(model.state_dict(), MODEL_PATH)
            print("✅ Best model saved!")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1/10] Train Loss: 1.4926 | Test Loss: 1.5395 | Acc: 56.55% | F1: 0.5684 | Precision: 0.6765 | Recall: 0.5655
✅ Best model saved!
[Epoch 2/10] Train Loss: 1.5703 | Test Loss: 1.5757 | Acc: 51.99% | F1: 0.5074 | Precision: 0.5470 | Recall: 0.5199
[Epoch 3/10] Train Loss: 1.5119 | Test Loss: 1.2668 | Acc: 67.03% | F1: 0.6636 | Precision: 0.6935 | Recall: 0.6703
✅ Best model saved!
[Epoch 4/10] Train Loss: 1.3411 | Test Loss: 1.1787 | Acc: 70.80% | F1: 0.7051 | Precision: 0.7335 | Recall: 0.7080
✅ Best model saved!
[Epoch 5/10] Train Loss: 1.2425 | Test Loss: 1.1014 | Acc: 73.81% | F1: 0.7317 | Precision: 0.7420 | Recall: 0.7381
✅ Best model saved!
[Epoch 6/10] Train Loss: 1.1586 | Test Loss: 1.0365 | Acc: 76.63% | F1: 0.7630 | Precision: 0.7728 | Recall: 0.7663
✅ Best model saved!
[Epoch 7/10] Train Loss: 1.0903 | Test Loss: 0.9882 | Acc: 78.41% | F1: 0.7875 | Precision: 0.7984 | Recall: 0.7841
✅ Best model saved!
[Epoch 8/10] Train Loss: 0.9971 | Test Loss: 0.9323 | Acc: 81.03% | 